In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from shapely.wkb import loads
from shapely import from_wkb, from_wkt
import pandas as pd
from IPython.display import display
from coordinate_to_geohash_conv import process_points_column
from binary_to_coordinate_conv import process_wkb_column

conf = SparkConf()
conf.setAll([
    ("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0"),
    ("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"),
    ("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
])

spark = SparkSession.builder \
    .master("local[*]") \
    .config(conf=conf) \
    .appName("read-s3-with-spark") \
    .getOrCreate()

df = spark.read \
    .option("compression", "zstd") \
    .parquet("s3a://overturemaps-us-west-2/release/2024-12-18.0/theme=divisions/type=division/part-00000-ed644172-af5f-489e-9488-f690fd04c569-c000.zstd.parquet")

df_tr = df.filter(df["country"] == "TR")

#df_tr.show(5, truncate=True)

25/01/31 01:14:55 WARN Utils: Your hostname, ubuntu-s-4vcpu-16gb-320gb-intel-fra1-01 resolves to a loopback address: 127.0.1.1; using 10.19.0.11 instead (on interface eth0)
25/01/31 01:14:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-68d868a0-f9ad-44a1-883e-c614413269a3;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 211ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------

In [3]:
# spark df sini pandas df sine çevirme

# Collect data as a list of rows
rows = df_tr.collect()

# Convert to Pandas DataFrame
pandas_df = pd.DataFrame(rows, columns=df_tr.columns)
#display(pandas_df)

In [4]:
trunc_pandas_df = pandas_df[['geometry','bbox','country']]
display(trunc_pandas_df)

,geometry,bbox,country
0,"[0, 0, 0, 0, 1, 64, 59, 60, 147, 239, 53, 173,...","(27.236631393432617, 27.236635208129883, 37.05...",TR
1,"[0, 0, 0, 0, 1, 64, 59, 62, 215, 198, 251, 210...","(27.245479583740234, 27.2454833984375, 37.0721...",TR
2,"[0, 0, 0, 0, 1, 64, 59, 70, 185, 102, 190, 122...","(27.27626609802246, 27.276269912719727, 37.000...",TR
3,"[0, 0, 0, 0, 1, 64, 59, 74, 106, 74, 247, 117,...","(27.29068374633789, 27.290687561035156, 36.967...",TR
4,"[0, 0, 0, 0, 1, 64, 59, 74, 98, 39, 60, 132, 2...","(27.290559768676758, 27.290563583374023, 36.97...",TR
...,...,...,...
81919,"[0, 0, 0, 0, 1, 64, 69, 211, 226, 69, 4, 22, 1...","(43.65534210205078, 43.65534973144531, 40.8280...",TR
81920,"[0, 0, 0, 0, 1, 64, 69, 210, 185, 80, 24, 70, ...","(43.64627456665039, 43.64628219604492, 40.8542...",TR
81921,"[0, 0, 0, 0, 1, 64, 69, 213, 127, 0, 145, 31, ...","(43.667938232421875, 43.66794204711914, 40.877...",TR
81922,"[0, 0, 0, 0, 1, 64, 69, 208, 34, 182, 13, 109,...","(43.62605285644531, 43.626060485839844, 40.942...",TR


In [5]:
display(trunc_pandas_df['geometry'])

0        [0, 0, 0, 0, 1, 64, 59, 60, 147, 239, 53, 173,...
1        [0, 0, 0, 0, 1, 64, 59, 62, 215, 198, 251, 210...
2        [0, 0, 0, 0, 1, 64, 59, 70, 185, 102, 190, 122...
3        [0, 0, 0, 0, 1, 64, 59, 74, 106, 74, 247, 117,...
4        [0, 0, 0, 0, 1, 64, 59, 74, 98, 39, 60, 132, 2...
                               ...                        
81919    [0, 0, 0, 0, 1, 64, 69, 211, 226, 69, 4, 22, 1...
81920    [0, 0, 0, 0, 1, 64, 69, 210, 185, 80, 24, 70, ...
81921    [0, 0, 0, 0, 1, 64, 69, 213, 127, 0, 145, 31, ...
81922    [0, 0, 0, 0, 1, 64, 69, 208, 34, 182, 13, 109,...
81923    [0, 0, 0, 0, 1, 64, 69, 206, 137, 96, 102, 114...
Name: geometry, Length: 81924, dtype: object

In [6]:
print(type(trunc_pandas_df['geometry']))
print(type(trunc_pandas_df['geometry'].iloc[0]))
print(type(trunc_pandas_df['geometry'].iloc[0][0]))

<class 'pandas.core.series.Series'>
<class 'bytearray'>
<class 'int'>


In [7]:
point_df = process_wkb_column(trunc_pandas_df, 'geometry', 'point')
display(point_df)

,geometry,bbox,country,point
0,"[0, 0, 0, 0, 1, 64, 59, 60, 147, 239, 53, 173,...","(27.236631393432617, 27.236635208129883, 37.05...",TR,POINT (27.2366323 37.0535073)
1,"[0, 0, 0, 0, 1, 64, 59, 62, 215, 198, 251, 210...","(27.245479583740234, 27.2454833984375, 37.0721...",TR,POINT (27.24548 37.0721781)
2,"[0, 0, 0, 0, 1, 64, 59, 70, 185, 102, 190, 122...","(27.27626609802246, 27.276269912719727, 37.000...",TR,POINT (27.2762665 37.0002005)
3,"[0, 0, 0, 0, 1, 64, 59, 74, 106, 74, 247, 117,...","(27.29068374633789, 27.290687561035156, 36.967...",TR,POINT (27.2906844 36.9675442)
4,"[0, 0, 0, 0, 1, 64, 59, 74, 98, 39, 60, 132, 2...","(27.290559768676758, 27.290563583374023, 36.97...",TR,POINT (27.2905602 36.9791803)
...,...,...,...,...
81919,"[0, 0, 0, 0, 1, 64, 69, 211, 226, 69, 4, 22, 1...","(43.65534210205078, 43.65534973144531, 40.8280...",TR,POINT (43.6553427 40.8281007)
81920,"[0, 0, 0, 0, 1, 64, 69, 210, 185, 80, 24, 70, ...","(43.64627456665039, 43.64628219604492, 40.8542...",TR,POINT (43.6462803 40.854264)
81921,"[0, 0, 0, 0, 1, 64, 69, 213, 127, 0, 145, 31, ...","(43.667938232421875, 43.66794204711914, 40.877...",TR,POINT (43.6679383 40.8771935)
81922,"[0, 0, 0, 0, 1, 64, 69, 208, 34, 182, 13, 109,...","(43.62605285644531, 43.626060485839844, 40.942...",TR,POINT (43.6260593 40.9425254)


In [8]:
print(type(point_df['point']))
print(type(point_df['point'].iloc[0]))
#print(type(point_df['point'].iloc[0][0]))
print(point_df['point'].iloc[0])
print(point_df['point'].iloc[0].x)
print(point_df['point'].iloc[0].y)

<class 'pandas.core.series.Series'>
<class 'shapely.geometry.point.Point'>
POINT (27.2366323 37.0535073)
27.2366323
37.0535073


In [9]:
precision = 7
geohash_df = process_points_column(point_df, 'point', 'geohash', precision)
display(geohash_df)

,geometry,bbox,country,point,geohash
0,"[0, 0, 0, 0, 1, 64, 59, 60, 147, 239, 53, 173,...","(27.236631393432617, 27.236635208129883, 37.05...",TR,POINT (27.2366323 37.0535073),swe6gc3
1,"[0, 0, 0, 0, 1, 64, 59, 62, 215, 198, 251, 210...","(27.245479583740234, 27.2454833984375, 37.0721...",TR,POINT (27.24548 37.0721781),swe6guz
2,"[0, 0, 0, 0, 1, 64, 59, 70, 185, 102, 190, 122...","(27.27626609802246, 27.276269912719727, 37.000...",TR,POINT (27.2762665 37.0002005),swe6kxt
3,"[0, 0, 0, 0, 1, 64, 59, 74, 106, 74, 247, 117,...","(27.29068374633789, 27.290687561035156, 36.967...",TR,POINT (27.2906844 36.9675442),swe6m18
4,"[0, 0, 0, 0, 1, 64, 59, 74, 98, 39, 60, 132, 2...","(27.290559768676758, 27.290563583374023, 36.97...",TR,POINT (27.2905602 36.9791803),swe6m5b
...,...,...,...,...,...
81919,"[0, 0, 0, 0, 1, 64, 69, 211, 226, 69, 4, 22, 1...","(43.65534210205078, 43.65534973144531, 40.8280...",TR,POINT (43.6553427 40.8281007),szr032s
81920,"[0, 0, 0, 0, 1, 64, 69, 210, 185, 80, 24, 70, ...","(43.64627456665039, 43.64628219604492, 40.8542...",TR,POINT (43.6462803 40.854264),szr03jq
81921,"[0, 0, 0, 0, 1, 64, 69, 213, 127, 0, 145, 31, ...","(43.667938232421875, 43.66794204711914, 40.877...",TR,POINT (43.6679383 40.8771935),szr099q
81922,"[0, 0, 0, 0, 1, 64, 69, 208, 34, 182, 13, 109,...","(43.62605285644531, 43.626060485839844, 40.942...",TR,POINT (43.6260593 40.9425254),szr0btr
